# Star Finder and Imexam

J. Runnoe <br>
Fall, 2021

This notebook will guide you through the steps necessary for detecting stars in your overscan subtracted, trimmed, bias subtracted, flat fielded science frames.

Doing aperture photometry requires that you be able to find stars in your image. Manually identifying the positions of stars in DS9 is not scalable to large numbers of stars or frames. Therefore, we will use the detection tools in PHOTUTILS. You can see example code and documentation here:

<a href="https://photutils.readthedocs.io/en/stable/detection.html">Star finder example code</a> <br>
<a href="https://photutils.readthedocs.io/en/stable/api/photutils.detection.DAOStarFinder.html#photutils.detection.DAOStarFinder">DAOStarFinder documentation</a> <br>
<a href="https://imexam.readthedocs.io/en/0.9.1/">imexam</a> <br>
<a href="https://imexam.readthedocs.io/en/latest/api/imexam.imexamine.Imexamine.html#imexam.imexamine.Imexamine.radial_profile">imexam radial profiles</a> <br>
<a href="https://github.com/spacetelescope/imexam">imexam repo with some example notebooks</a> <br>

---
## Contents
* [Setup](#setup)
    * [Download imexam](#imexam)
* [Exercises](#exercises)
* [Summary](#summary)

## Setup <a class="anchor" id="setup"></a>

### Download imexam <a class="anchor" id="imexam"></a>

Imexam is a handy piece of software that traditionally worked well with DS9. These days, it is implemented in Python but getting the windows working is something of a nightmare. However, you can use some of the imexam tools without getting its full implementation! So I recommend installing it regardless.

You can get imexam with pip (first make sure you're in the class Anaconda environment):<br>
`$ pip install imexam` <br>

Add the directory for DS9 to your path. For example, I added:<br>
`/Volumes/Colfax/Applications/SAOImageDS9.app/Contents/MacOS/ds9`<br>
to my PATH and PYTHON_PATH variables (for good measure).

---
## Exercises <a class="anchor" id="exercises"></a>


In [62]:
# import block
# some of these may not be needed
import numpy as np
from astropy.io import fits
import pdb
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import rc
import matplotlib
from astropy.visualization import hist
from datetime import datetime
from pathlib import Path
import os
from ccdproc import ImageFileCollection
import ccdproc as ccdp
from astropy.modeling.models import Polynomial1D,Chebyshev1D,Legendre1D,Hermite1D, Gaussian1D,Gaussian2D,Polynomial2D
from astropy.modeling import fitting
from astropy.stats import mad_std
from astropy.nddata import CCDData, StdDevUncertainty
from scipy import stats
from astropy import units as u
from photutils.aperture import CircularAperture,CircularAnnulus
from photutils.aperture import aperture_photometry
from astropy.stats import sigma_clipped_stats
from photutils.detection import DAOStarFinder
from astropy.table import Table
from imexam.imexamine import Imexamine
from astropy.coordinates import SkyCoord

In [2]:
# import convenience plotting functions downloaded from 
# here: https://github.com/mwcraig/ccd-reduction-and-photometry-guide
phot_tutorial_dir = '/Users/runnojc1/Software/py/ccd-reduction-and-photometry-guide/notebooks/'
import sys
sys.path.insert(0,phot_tutorial_dir)
from convenience_functions import show_image

In [3]:
# plotting defaults
# Use custom style for larger fonts and figures
plt.style.use(phot_tutorial_dir+'/guide.mplstyle')

# Set some default parameters for the plots below
rc('font', size=12)
rc('axes', grid=True)

In [4]:
data_dir = '/Users/runnojc1/Dropbox/Research/teaching/F2021/ASTR8060/Imaging/'            # raw data directory
reduced_dir = '/Users/runnojc1/Dropbox/Research/teaching/F2021/ASTR8060/Imaging_reduced/' # reduced working directory

#### 1. Find bright stars in the middle of your frame

The star finder implemented in PHOTUTILS is an implementation of <a href="https://ui.adsabs.harvard.edu/abs/1987PASP...99..191S/abstract">DAOPHOT</a>. Generally speaking, it looks for peaks in the distribution of pixel values in a frame. The challenges are to detect real stellar peaks near the level of the background and/or noise in the image, and to distinguish stellar peaks from various kinds of noise spikes and other astronomical sources (e.g., galaxies).


##### (a) Look at the documentation and DAOPHOT paper to answer the following:
 - How do you control how faint the stars you find are? <br>

 - How do you avoid saturated stars?<br>

 - What is sharpness?<br>

 - What is roundness?<br>

##### (b) Read in the data and plot the image

Plot the central part of the frame for columns 750-1250 and rows 1000-1500 for a085otzf.fits.

##### (c) Find the bright stars
Look at the Star Finder example code to see the syntax for this. You will need a guess for fwhm (we'll measure this next). Approximately how many pixels would be reasonable starting guess? (The size of a faint start might be a good starting place.)

##### (d) Plot the image again and mark the detected stars

Making apertures at the position of the stars you detect would be a good method for showing where they are.

#### 2. Measure the PSF FWHM

The point spread function (PSF) full width at half maximum (FWHM) is a measure of the width in pixels where light from a point source is distributed on the CCD. Many of the detection methods above take the PSF FWHM as an input to help distinguish from noise spikes. Also, you may want to explore the PFS as a function of time or position on the CCD. Thus, it is worthwhile to know how to measure it.

A simple estimate for the PSF FWHM would find the 1D Gaussian distribution of light from a single star in a column or row and determine its width. More complex algorithms include centroiding, 2D model fitting, and background subtraction. The imexam package includes these and many more tools. 

(Imexam also has many interactive tools and can run with DS9, but it is finicky to get the display set up. See the accompanying notebook if you are feeling extremely bold.)


We will use imexam to create a radial profile plot. This shows the azimuthally averaged value of the counts in a frame as a function of radius from the star centroid. The FWHM is taken from the width of this (Gaussian) profile.

A warning about the radial profiles in imexam: they can fail with various errors if the position is near the edge of the frame.

##### (a) Find the FWHM for the sources you found in the previous step.

To do this, we will use imexam to fit a 1D Gaussian function, and read the FHWM values off of the plots. 

As a point of interest, you can also return the data from these plots to fit your own 1D function and return a numerical value for the FHWM if you are so inclined. The code to return the data looks like this.
```python

data = radial_profile(1324,1054,data,genplot=False)
radius = data[0]
flux = data[1]
```

In [10]:
#initalize the object to get access to the plotting functions
plots=Imexamine() 

In [1]:
# here is one radial profile at 1325,1054
plots.radial_profile(1325,1054,data,genplot=True)

# loop over the x and y centroids from the star finder
# and generate a radial profile for each one


#### 3. Write a function to do aperture photometry

Write a function that takes a filename and:

1. Reads in the data.
2. Finds all the sources in the image.
3. Puts a circular aperture and background annulus on each of the stars.
4. Calculates background-subtracted instrumental magnitudes.
5. Plots the image and apertures for visual inspection.
6. Returns a table of photometry, including the instrumental magnitudes.

Run your function on a085otzf.fits.

In [68]:
def get_phot(filename):
    # do photometry
    phot = 0
    return phot

In [2]:
phot = get_phot(reduced_dir+'a085otzf.fits')
phot

---
## Summary <a class="anchor" id="summary"></a>

At this point, all of you should:
* Be able to find star positions automatically.
* Estimate the FWHM of stars in your image.
* Do aperture photometry at a list of positions.